# K3D - Release 2.1.0

Final tests




 
  

In [ ]:
import k3d
k3d.version_info

In [ ]:
import numpy as np

plot = k3d.plot()
plot.display()

plot += k3d.vectors(
   (0,0,0,0,0,0,0,0,0), 
   (1,0,0,0,1,0,0,0,1), 
    colors=(0xff0000, 0xff0000, 0xffff00, 0xffff00, 0x00ff00, 0x00ff00), 
   labels=('x', 'y', 'z')
)



## surface 

 
 



In [ ]:
np.random.randint(0, 0xFFFFFF, 1)[0].dtype

In [ ]:
plot.grid_auto_fit =  True

In [ ]:
Nx = 50
Ny = 50
xmin,xmax = -3,3
ymin,ymax = -0,3


x = np.linspace(xmin,xmax,Nx)
y = np.linspace(ymin,ymax,Ny)
x,y = np.meshgrid(x,y)

surface_plot =  k3d.surface(np.sin(x**2+y**2),xmin=xmin,xmax=xmax,\
                    ymin=ymin,ymax=ymax, color=int(np.random.randint(0, 0xFFFFFF, 1)[0]))
plot += surface_plot

In [ ]:
%%time 


Nx,Ny = 40,50
dx,dy = 1.4,2.7
x0,y0 = .2,1.3
xmin,xmax = x0-dx, x0+dx
ymin,ymax = y0-dy, y0+dy
x = np.linspace(xmin,xmax,Nx)
y = np.linspace(ymin,ymax,Ny)
x,y = np.meshgrid(x,y)
surface_plot2 = k3d.surface(np.sin(x**2+y**2),xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax, color=int(np.random.randint(0, 0xFFFFFF, 1)[0]))
plot += surface_plot2


## text



In [ ]:
texture_text = k3d.texture_text("K3D.text(\"X max\", position=(3, 0, 0) )", position=(0, 0, 1), size=10)
plot += texture_text

In [ ]:
text = k3d.text("O", position=(0, 0, 1) )
plot += text

In [ ]:
text.text = "\int_0^1 dx \sin x"

In [ ]:
import time
for i in range(26):
    text.position = [0,0,1+np.sin(0.1*i)]
    time.sleep(0.1)

## points

In [ ]:
x = x.flatten()[::4]
y = y.flatten()[::4]
z = np.sin(x**2+y**2) 
points = np.vstack([x,y,z]).T 
points_plt = k3d.points(points,colors=np.ones_like(x)*0xFF0F00 ,point_size=.1)
plot += points_plt

## line 

In [ ]:
Nx,Ny = 10,65

In [ ]:
x = np.linspace(-3,0,Nx)
y = np.linspace(-3,0,Ny)
x, y = np.meshgrid(x,y)

for i in range(0,Nx):
    x_ = x[:,i]
    y_ = y[:,i]
    z = np.sin(x_**2+y_**2)
    points = np.vstack([x_,y_,z]).T
    vars()['line_%d'%i]  = k3d.line(points,color=0xFF0000 ,width=1)
    plot += vars()['line_%d'%i]

## vectors

In [ ]:
Nx,Ny = 100,100
x = np.linspace(xmin,xmax,Nx)
y = np.linspace(ymin,ymax,Ny)
dx, dy  = x[1]-x[0],y[1]-y[0]

x,y = np.meshgrid(x,y)
z = np.sin(x**2+y**2)
dx,dy 
if True:
    dFy,dFx = np.gradient(z)
    dFy,dFx = dFy/dy,dFx/dx 
else:
    dFx,dFy = 2*x*np.cos(x**2+y**2),2*y*np.cos(x**2+y**2)

origins = np.vstack([x.flatten(),y.flatten(),z.flatten()]).T
vectors = 0.01*(np.vstack([dFx.flatten(),dFy.flatten(),-np.ones(Nx*Ny)] )).T


In [ ]:
def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)

In [ ]:
vector_plot = k3d.vectors(-0.4*normalized(vectors[::10]), origins[::10], color=0xffff00)
plot += vector_plot

#  marching cubes



In [ ]:
Nz = Nx
zmin,zmax = -1,1
ymin,ymax = 2,6

x = np.linspace(xmin,xmax,Nx//2)
y = np.linspace(ymin,ymax,Ny//2)
z = np.linspace(zmin,zmax,Nz//2)
x,y,z = np.meshgrid(x,y,z)
F = np.sin(x**2+y**2)-z

marching_cubes =k3d.marching_cubes(np.moveaxis(F,2,0),\
          xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax, \
          zmin=zmin, zmax=zmax, level=0.0,color=0xff0000)
plot += marching_cubes

In [ ]:
plot.camera_auto_fit = False

In [ ]:
from ipywidgets import widgets

class plot_ctrl(object):
    plot = None
    def __init__(self,id1,plot):
        self.id = id1
        ref_names = [ n for n,v in globals().items() if self.id==id(v)]  
        if len(ref_names)==1:
            desc = ref_names[0]
        else:
            desc = str(self.id)
        self.w = widgets.Checkbox(description=desc,value=True)
        self.w.observe(self.runme)
        if type(self).plot == None:
            type(self).plot = plot
            type(self).objs = plot.objects.copy()

    def runme(self,change):
        if change['name']=='value':
            for obj in type(self).objs:
                if obj.id == self.id: #int(self.w.description):
                    if self.w.value==True:
                        type(self).plot += obj
                        #print("Added obj")
                    else:
                        type(self).plot -= obj  
                        #print("Removed obj")
    
    @classmethod
    def VBox(cls,plot,ncol = 4):
        names = {obj.id:obj.__class__.__name__ for obj in plot.objects}
        cls.l = [cls(id1=key,plot=plot) for key in names.keys()]
        
        if ncol < len(cls.l):
            checkboxes = widgets.VBox( [widgets.HBox([ el.w for el in cls.l[i:i+ncol]]) for i in range(0,len(cls.l),ncol)] ) 
        else:
            checkboxes = widgets.VBox([el.w for el in cls.l])
        return widgets.VBox([checkboxes,plot])
    
plot_ctrl.VBox(plot)